Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [1]:
#import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

#with open('enrollments.csv', 'rb') as f:
#    reader = unicodecsv.DictReader(f)
#    enrollments = list(reader)

In [2]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.
# The function that can takes three csv files at one 

import unicodecsv

def read_csv(filename):
    with open (filename,'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)
enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')

print enrollments[0]

{u'status': u'canceled', u'is_udacity': u'True', u'is_canceled': u'True', u'join_date': u'2014-11-10', u'account_key': u'448', u'cancel_date': u'2015-01-14', u'days_to_cancel': u'65'}


In [3]:
print daily_engagement[0]

{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}


In [4]:
print project_submissions[0]

{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


## Fixing Data Types

In [5]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given (student hasn't cancelled yet), returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [6]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{u'acct': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [7]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [8]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

In [9]:
len(enrollments)

1640

In [10]:
unique_enrolled_students = set()
# The student can enroll, and then cancel, and then re-enroll later. 
# But their account key is the same. 
for enrollment in enrollments:
    unique_enrolled_students.add(enrollment['account_key'])
len(unique_enrolled_students)

1302

In [11]:
len(daily_engagement)

136240

In [12]:
unique_engagement_students = set()

for engagement_record in daily_engagement:
    unique_engagement_students.add(engagement_record['acct'])
len(unique_engagement_students)

1237

In [13]:
len(project_submissions)

3642

In [14]:
unique_project_submitters = set()
for submission in project_submissions:
    unique_project_submitters.add(submission['account_key'])
len(unique_project_submitters)

743

## Problems in the Data

There are two problems in the data

1. More unique students in enrollment than engagement table 

2. Column named *account_key* in two tables and *acct* in the third which brings some inconvenience. 

In [15]:
#####################################
#                 3                 #
#####################################

## Fix prob #2: ##
# Rename the "acct" column in the daily_engagement table
# to "account_key".

for engagement_record in daily_engagement:
    engagement_record['account_key'] = engagement_record['acct']
    del[engagement_record['acct']]

Now we can update the previous section about investigating data

In [16]:
# Create a function that will sort the dataset by 'account_key'
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students

In [17]:
# Number of total enrolled students 
len(enrollments)

1640

In [18]:
# Number of unique students enrolled
unique_enrolled_students = get_unique_students(enrollments)

In [19]:
len(unique_enrolled_students)

1302

In [20]:
# Number of total engaged students 
len(daily_engagement)

136240

In [21]:
# Number of unique engaged students
unique_engagement_students = get_unique_students(daily_engagement)
len(unique_engagement_students)

1237

In [22]:
# Number of total project submitters 
len(project_submissions)

3642

In [23]:
# Number of unique project submitters
unique_project_submitters = get_unique_students(project_submissions)
len(unique_project_submitters)

743

## Missing Engagement Records

Why are students missing from *daily_engagement* ?

1. Identify surprising data points
    - any enrollment record with no correspondary engagement data
2. Print out one or a few surprising data points
3. Fix any problem you find
    - More investigation may be necessary
    - Or there might not be a problem
    

In [24]:
#####################################
#                 4                 #
#####################################

## Fix prob #1: 
## Find any one student enrollments where the student is missing 
## from the daily engagement table.
## Output that enrollment.

for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students:
        print enrollment
        break

{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 12, 0, 0), u'account_key': u'1219', u'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), u'days_to_cancel': 0}


In this case, the student (acct:1219) had cancelled within one day of joining, which explains why his/her record was not in the engagement table. 

We repeat the investigation process, jumping back to step 1 to find more surprising data points, if any. 

## Checking for More Problem Records

In [25]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.

# Create a variable storing the number of problem students
num_problem_students = 0
# Create a for loop checking whether that account key was present in the daily engagement table
for enrollment in enrollments:
    student = enrollment['account_key']
    if (student not in unique_engagement_students and enrollment ['join_date'] != enrollment['cancel_date']):
        print enrollment
        num_problem_students += 1
num_problem_students

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}


3

## Tracking Down the Remaining Problems

In [26]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [27]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [28]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634


## Refining the Question

**Question:** 

How do numbers in the daily engagement table differ for students who pass the first project?

**Problem:** 
1. This will include data from after the project submission .
2. This compares data from different lengths of time. 
3. Includes engagement in courses not related to the first project. 

**Revision:** 

Only look at engagement from first week, and exclude students who canceled within a week.

**Getting started:**

Create a dictionary of students who either:
- haven't cancelled yet (days_to_cancel is None)
- stayed enrolled more than 7 days (days_to_cancel > 7) 

*keys*: account keys 

*values*: enrollment date

*Name your dictionary*: paid_student 

**Refined Question:**

How many students are in the dictionary? **

In [29]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students ={}
for enrollment in non_udacity_enrollments:
    if (not enrollment['is_canceled'] or 
            enrollment['days_to_cancel'] > 7):
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
# Problem: one student can enroll in multiple times 
# save the most recent enrollment date for those students 
# only add the enrollment date into dictionary if either:         
        if (account_key not in paid_students or # the account key was not already present 
                enrollment_date > paid_students[account_key]): # or if this enrollment date is more recent than the date that's already present 
                # key = account_key; value = enrollment_date
            paid_students[account_key] = enrollment_date

len(paid_students)

995

## Getting Data from First Week

In [30]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    # return time_delta.days <7 
    # The above code causes some issues 
    # Because it doesn't check the engagement data coming after the join date 
    # So for students who enrolled once, then cancel, then enroll again later, everything from the first enrollment will count as their first week
    # The below is the solution: checking at least zero days had passed from the join date to the engagement date
    # That way, we only consider data from most recent enrollment 
    
    return time_delta.days >= 0 and time_delta.days < 7

In [31]:
#####################################
#                 7                 #
#####################################

## Create a function to remove any data points corresponding to students who cancelled during the free trail
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point ['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submission = remove_free_trial_cancels(non_udacity_submissions)

print len(paid_enrollments)
print len(paid_engagement)
print len(paid_submission)

1293
134549
3618


Add the **has_visited** field for Number of visits in first week

In [46]:
for engagement_record in paid_engagement:
    if engagement_record['num_courses_visited']>0:
        engagement_record['has_visited'] = 1
    else:
        engagement_record['has_visited'] = 0

In [47]:
## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.
## Return the number of engagement records coming from the first week
paid_engagement_in_first_week = []

for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']

    # Check if the two dates are within one week of each other
    # append the record to the variable. 
    if within_one_week(join_date, engagement_record_date):
        paid_engagement_in_first_week.append(engagement_record)
        
len(paid_engagement_in_first_week)        

6919

## Exploring Student Engagement

In [48]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [49]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [50]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = total_minutes_by_account.values()
print 'Mean:', np.mean(total_minutes)
print 'Standard deviation:', np.std(total_minutes)
print 'Minimum:', np.min(total_minutes)
print 'Maximum:', np.max(total_minutes)

Mean: 306.708326753
Standard deviation: 412.996933409
Minimum: 0.0
Maximum: 3564.7332645


##### Debugging Data Analysis Code

1. Identify surprising data points
2. Print out one or a few surprising points
3. Fix any problems you find

In [51]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

# Figure out the problem of students with maximum number of minute 
student_with_max_minutes = None
max_minutes = 0

for student, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes # reset max min
        student_with_max_minutes = student

max_minutes

3564.7332644989997

In [52]:
for engagement_record in paid_engagement_in_first_week:
    if engagement_record['account_key'] == student_with_max_minutes:
        print engagement_record

{u'lessons_completed': 4, u'num_courses_visited': 4, 'has_visited': 1, u'total_minutes_visited': 850.519339666, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 9, 0, 0)}
{u'lessons_completed': 6, u'num_courses_visited': 6, 'has_visited': 1, u'total_minutes_visited': 872.633923334, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 10, 0, 0)}
{u'lessons_completed': 6, u'num_courses_visited': 2, 'has_visited': 1, u'total_minutes_visited': 777.018903666, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 11, 0, 0)}
{u'lessons_completed': 2, u'num_courses_visited': 1, 'has_visited': 1, u'total_minutes_visited': 294.568774, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 12, 0, 0)}
{u'lessons_completed': 1, u'num_courses_visited': 3, 'has_visited': 1, u'total_minutes_visited': 471.2139785, u'projects_completed': 0, 'account_key': u'163

## Lessons Completed in First Week

In [53]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

# Function to group the record by account_key (same as #43)
# @data = arument 1
# @key_name = the key to group by
# @return = dictionary mapping account keys to lists of records
def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key].append(data_point)
    return grouped_data

In [54]:
# Call the group_data function to create the same engagement by account dictionary in 33

engagement_by_account = group_data(paid_engagement_in_first_week,
                                   'account_key')

In [55]:
# Function to sum up all the entries for each account_key 
# @ arg1: inputting the grouped data 
# @ arg2: field name (investigating on both the total min visited and lessons completed)
def sum_grouped_items(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            total += data_point[field_name]
        summed_data[key] = total
    return summed_data

total_minutes_by_account = sum_grouped_items(engagement_by_account,
                                             'total_minutes_visited')


In [56]:
# Function to print out summary statistics like the mean, and std dev
def describe_data(data):
    print 'Mean:', np.mean(data)
    print 'Standard deviation:', np.std(data)
    print 'Minimum:', np.min(data)
    print 'Maximum:', np.max(data)
    
describe_data(total_minutes_by_account.values())


Mean: 306.708326753
Standard deviation: 412.996933409
Minimum: 0.0
Maximum: 3564.7332645


In [57]:
lessons_completed_by_account = sum_grouped_items(engagement_by_account,
                                                 'lessons_completed')
describe_data(lessons_completed_by_account.values())

Mean: 1.63618090452
Standard deviation: 3.00256129983
Minimum: 0
Maximum: 36


## Number of Visits in First Week

In [58]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.
days_visited_by_account = sum_grouped_items(engagement_by_account, 'has_visited')
describe_data(days_visited_by_account.values())

Mean: 2.86733668342
Standard deviation: 2.25519800292
Minimum: 0
Maximum: 7


## Splitting out Passing Students

In [61]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

pass_subway_project = set()

for submission in paid_submission:
    project = submission['lesson_key']
    rating = submission['assigned_rating']
    
    if ((project in subway_project_lesson_keys) and 
               (rating == 'PASSED' or rating == 'DISTINCTION')):
        pass_subway_project.add(submission['account_key'])
    
len(pass_subway_project)

647

In [62]:
passing_engagement = []
non_passing_engagement =[]

for engagement_record in paid_engagement_in_first_week:
    if engagement_record['account_key'] in pass_subway_project:
        passing_engagement.append(engagement_record)
    else:
        non_passing_engagement.append(engagement_record)

        print len(passing_engagement)
print len(non_passing_engagement)

4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527
4527


## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.